# 第9章 顺序容器

- 一个容器就是一些特定类型对象的集合，顺序容器（sequential container）为程序员提供了控制元素存储和访问顺序的能力，这种顺序不依赖于元素的值，而是与元素加入容器时的位置相对应

## 9.1 顺序容器概述

- 标准库中的顺序容器有：vector、deque、list、forward_list、array、string；所有顺序容器都提供了快速顺序访问元素的能力，但是它们在以下方面都有不同的性能折中：1.向容器添加或从容器中删除元素的代价；2.非顺序访问容器中元素的代价
- 现代C++程序应该使用标准库容器，而不是更原始的数据结构，如内置数组
- 通常，使用vector是最好的选择，除非你有很好的理由选择其他容器

## 9.2 容器库概览

- 容器类型上的操作形成了一种层次：某些操作是所有容器类型都提供的，另外一些操作仅针对顺序容器、关联容器或无序容器，还有一些操作只适用于一小部分容器
- 在本节中我们将介绍对所有容器都适用的操作

### 9.2.1 迭代器

- 与容器一样，迭代器有着公共的接口：如果一个迭代器提供某个操作，那么所有提供相同操作的迭代器对这个操作的实现方式都是相同的

### 9.2.2 容器类型成员

- 每个容器都定义了多个类型，我们已经使用过其中三种：size_type、iterator、const_iterator；除此之外，还有反向迭代器以及一些在泛型编程中使用的类型别名

### 9.2.3 begin和end成员

- begin和end有多个版本：带r的版本返回反向迭代器，以c开头的版本则返回const迭代器

### 9.2.4 容器定义和初始化

- 将一个新容器创建为另一个容器的拷贝的方法有两种：可以直接拷贝整个容器，或者（array除外）拷贝由一个迭代器对指定的元素范围；对第一种方式，要求两个容器的类型及其元素类型必须匹配，对于第二种方式，不要求容器类型相同，甚至也不要求元素类型相同，只要能将要拷贝的元素转换为要初始化的容器的元素类型即可

### 9.2.5 赋值和swap

#### 使用assign

- 赋值运算符要求左边和右边的运算对象具有相同的类型，它将右边运算对象中所有元素拷贝到左边运算对象中
- 顺序容器（array除外）还定义了一个名为assign的成员，允许我们从一个不同但相容的类型赋值，或者从容器的一个子序列赋值，assign操作用参数所指定的元素（的拷贝）替换左边容器中的所有元素

```c++
list<string> names;
vector<const char*> oldstyle;
names = oldstyle; // 错误：容器类型不匹配
names.assign(oldstyle.cbegin(), oldstyle.cend()); // 正确：可以将const char*转换为string

// assign的第二个版本，接受一个整型值和一个元素值，用指定数目且具有相同给定值的元素替换容器中原有元素
list<string> slist1(1); 
slist.assign(10, "Hiya!");
```

#### 使用swap

- swap操作交换两个相同类型容器的内容，调用swap之后，两个容器中的元素将会交换
```c++
vector<string> svec1(10);
vector<string> svec2(24);
swap(svec1, svec2);
```
- 除array外，swap只是交换两个容器的内部结构，操作很快；并且，指向容器的迭代器、引用和指针在swap操作之后都不会失效，它们仍指向swap操作之前所指向的那些元素，但是，在swap之后，这些元素已经属于不同的容器了，例如，假定iter在swap之前指向svec1\[3\]的string，那么在swap之后它指向svec2\[3\]的元素
- 与其他容器不同，对一个string调用swap会导致迭代器、引用和指针失效
- 对于array，swap会真正交换它们的元素，因此耗时和array的元素数量成正比，并且swap之后，指针、引用和迭代器所绑定的元素保持不变，但元素值已经与另一个array中对应元素的值进行了交换

### 9.2.6 容器大小操作

- 除了forward_list，每个容器类型都有三个与大小相关的操作：成员函数size返回容器中元素的数目，empty当size为0时返回true，否则返回false，max_size返回一个大于或等于该类型容器所能容纳的最大元素的值
- forward_list不支持size

### 9.2.7 关系运算符

- 每个容器类型都支持相等运算符（==和!=）；除了无序关联容器外的所有容器都支持关系运算符（>, >=, <, <=）
- 关系运算符左右两边的运算对象必须是相同类型的容器，且必须保存相同类型的元素
- 容器的关系运算符使用元素的关系运算符完成比较，因此只有当其元素类型也定义了相应的比较运算符时，我们才可以使用关系运算符来比较两个容器

## 9.3 顺序容器操作

- 顺序容器和关联容器的不同之处在于两者组织元素的方式，这些不同之处直接关系到了元素如何存储、访问、添加以及删除

### 9.3.1 向顺序容器添加元素

- 除array外，所有标准库容器都提供灵活的内存管理，在运行时可以动态添加或删除元素来改变容器大小
- 当我们使用这些操作时，必须记得不同容器使用不同的策略来分配元素空间，而这些策略直接影响性能；在一个vector或string的尾部之外的任何位置，或是一个deque的首尾之外的任何位置添加元素，都需要移动元素；而且，向一个vector或者string添加元素可能引起整个对象存储空间的重新分配，重新分配一个对象的存储空间需要分配新的内存，并将元素从旧的空间移动到新的空间中

#### 使用push_back

- 由于string是一个字符容器，我们可以用push_back在string末尾添加字符
```c++
string word;
word.push_back('s');
```
- 当我们用一个对象来初始化容器时，或将一个对象插入到容器中时，实际上放入到容器中的是对象值的一个拷贝，而不是对象本身，容器中的元素与提供值的对象之间没有任何关联，随后对容器中元素的任何改变都不会影响到原始对象，反之亦然

#### insert

- 在新标准下，接受元素个数或范围的insert版本返回指向第一个新加入的元素的迭代器（旧版本中返回void），如果范围为空，不插入任何元素，insert操作会将第一个参数返回

#### emplace

- 新标准引入了三个新成员——emplace_front、emplace和emplace_back，这些操作构造而不是拷贝元素，它们分别对应push_front、insert和push_back，允许我们将元素放置在容器头部、一个指定位置之前或容器尾部
- 当调用push或insert成员函数时，我们将元素类型的对象传递给它们，这些对象被拷贝到容器中；而当我们调用一个emplace成员函数时，则是将参数传递给元素类型的构造函数，emplace成员使用这些参数在容器管理的内存空间中直接构造元素
```c++
// 使用三个参数的Sales_data构造函数
c.emplace_back("978-0590", 25, 15.99);
// 错误
c.push_back("978-0590", 25, 15.99);
// 正确：创建一个临时的Sales_data对象传递给push_back
c.push_back(Sales_data("978-0590", 25, 15.99));
```

### 9.3.2 访问元素

#### 访问成员函数返回的是引用

- 在容器中访问元素的成员函数（即，front、back、下标和at）返回的都是引用；如果容器是一个const对象，则返回值是const引用，否则返回的是普通引用，可以用来改变元素的值
```c++
// 对一个空容器调用front和back，就像使用一个越界的下标一样，严重错误
if (!c.empty()) {
    c.front() = 42; // 将42赋予c中的第一个元素
    auto &v = c.back(); // 如果使用auto，则必须显示定义为引用类型
    v = 1024;
    auto v2 = c.back(); // 不是引用
    v2 = 0;
}
```

#### 下标操作和安全的随机访问

- 下标运算符并不检查下标是否在合法范围内，使用越界的下标是一种严重的程序设计错误，而且编译器不检查这种错误
- 如果我们希望确保下标是合法的，可以使用at成员函数，它类似下标运算符，但如果下标越界，at会抛出一个out_of_range异常

### 9.3.3 删除元素

- pop_front和pop_back成员函数分别删除首元素和尾元素，返回void，因此如果你需要弹出的元素的值，就必须在执行弹出操作之前保存它
- 成员函数erase从容器中指定位置删除元素，我们可以删除由一个迭代器指定的单个元素，也可以删除由一对迭代器指定的范围内的所有元素，两种形式的erase都返回指向删除的（最后一个）元素之后位置的迭代器

### 9.3.5 改变容器大小

- 可以用resize来增大或缩小容器
```c++
list<int> ilist(10, 42); // 10个int：每个的值都是42 
ilist.resize(15); // 将5个值为0的元素添加到ilist的末尾
ilist.resize(25, -1); // 将10个值为-1的元素添加到ilist的末尾
ilist.resize(5); // 从ilist末尾删除20个元素
```
- resize操作接受一个可选的元素值参数，用来初始化添加到容器中的元素，如果调用者未提供此参数，新元素进行值初始化，如果容器保存的是类类型元素，且resize向容器添加新元素，则我们必须提供初始值，或者元素类型必须提供一个默认构造函数

### 9.3.6 容器操作可能使迭代器失效

- 向容器中添加元素和从容器中删除元素的操作可能会使指向容器元素的指针、引用或迭代器失效，一个失效的指针、引用或迭代器将不再表示任何元素，使用失效的指针、引用或迭代器是一种严重的程序设计错误，很可能引起与使用未初始化指针一样的问题

## 9.4 vector对象是如何增长的

- 为了支持快速随机访问，vector将元素连续存储
- 为了减少容器空间重新分配次数的策略，当不得不获取新的内存空间时，vector和string的实现通常会分配比新的空间需求更大的内存空间，容器预留这些空间作为备用，可用来保存更多的新元素，这样就不需要每次添加新元素都重新分配容器的内存空间了
- 容器的size是指它已经保存的元素的数目，而capacity则是在不分配新的内存空间的前提下它最多可以保存多少元素
- 实际上，只要没有操作需求超出vector的容量，vector就不能重新分配内存空间；只有在执行insert操作时size与capacity相等，或调用resize或reserve时给定的大小超过当前capacity，vector才可能重新分配内存空间，会分配多少超过给定容量的额外空间，取决于具体实现
- 虽然不同的实现可以采用不同的分配策略，但所有实现都应遵循一个原则：确保用push_back向vector添加元素的操作有高效率

## 9.5 额外的string操作

- substr操作返回一个string，它是原始string的一部分或全部的拷贝，可以传递给substr一个可选的开始位置和计数值
- 

## 9.6 容器适配器

- 除了顺序容器外，标准库还定义了三个顺序容器适配器：stack、queue和priority_queue；适配器（adapter）是标准库中的一个通用概念，容器、迭代器和函数都有适配器，本质上，一个适配器是一种机制，能使某种事物的行为看起来像另外一种事物一样；一个容器适配器接受一种已有的容器类型，使其行为看起来像一种不同的类型，例如，stack适配器接受一个顺序容器（除array或forward_list外），并使其操作起来像一个stack一样
- 每个适配器都在其底层顺序容器类型之上定义了一个新的接口